In [10]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
import warnings
warnings.filterwarnings("ignore")
import time
from sklearn import preprocessing
import python_speech_features as mfcc
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME="b2.wav"

def calculate_delta(array):
    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            deltas[i]=deltas[i]+j*(array[(i+j)%(rows)]-array[(rows+i-j)%(rows)])
            j+=1
        deltas[i] = deltas[i]/10
    return deltas

def extract_features(audio,rate):
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft=2058,appendEnergy = True)    
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)
    combined = np.hstack((mfcc_feat,delta)) 
    return combined

#path to training data
modelpath = "C:\\Users\\Prashu Gupta\\Downloads\\datasets\\Final_dataset"
gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]

models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
speakers   = [fname.split("\\")[-1].split(".gmm")[0] for fname 
              in gmm_files]

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()
sr,audio=read('C:\\Users\\Prashu Gupta\\Downloads\\datasets\\Final_dataset\\prashu.wav')

#Load the Gaussian gender Models
models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
speakers   = [fname.split("\\")[-1].split(".gmm")[0] for fname 
              in gmm_files]

vector   = extract_features(audio,sr)
log_likelihood = np.zeros(len(models)) 
    
for i in range(len(models)):
    gmm    = models[i]  #checking with each model one by one
    scores = np.array(gmm.score(vector))
    log_likelihood[i] = scores.sum()
    print(speakers[i],log_likelihood[i])
winner = np.argmax(log_likelihood)
print ("\tdetected as - ", speakers[winner])
time.sleep(1.0)

Abhay6.wav -26.435110598443966
ballu6.wav -25.84090470715281
prashu6.wav -19.392352546722357
ram6.wav -28.99838958041658
sonu6.wav -29.273197242490035
	detected as -  prashu6.wav
